In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from genparse import Float, CFGLM
from genparse.cfglm import locally_normalize, EOS, ERROR
from genparse.align.trie import TokenTrieApproximation, NextTokenTrie
from genparse.lm import GreedilyTokenizedLLM
from genparse.util import LarkStuff, display_table
from genparse.inference import TraceSWOR

In [ ]:
llm = GreedilyTokenizedLLM("gpt2")

In [ ]:
prompt = 'Hello my name is'
pp = llm.p_next(prompt, 10).normalize()
pp

In [ ]:
# create a character-level trie for the next token given the prompt
#NextTokenTrie(pp).show()

In [ ]:
pcfg = CFGLM(locally_normalize(LarkStuff(r"""

start: /[ ]*(|Mr\.[ ]|Dr\.[ ])Tim(othy)?[ ](|Fabbri[ ])Vieira(|,[ ]Ph\.D\.)/

""").char_cfg(.99), tol=1e-100))

In [ ]:
print(repr(''.join(pcfg.sample())))

In [ ]:
token_trie_approx = TokenTrieApproximation(llm, pcfg)
tracer = TraceSWOR()
for _ in range(1):
    with tracer:
        print('----------------------------------')
        print(tracer.root.mass)
        ys = token_trie_approx.sample(prompt, max_tokens=50, draw=tracer, verbosity=1)
        print(ys)

In [ ]:
tracer.root.graphviz(fmt_node=lambda x: f'{x._mass:.3g}', fmt_edge=lambda i,a,j: repr(a))

In [ ]:
fruit = CFGLM(locally_normalize(LarkStuff(r"""
start: (|" ") sentence

sentence: noun verb noun        
        | noun verb "like" noun 

noun: det* adj* NOUN
verb: VERB
adj: ADJ
det: "a" | "the"

NOUN: "flies" | "banana" | "fruit"
VERB: "like" | "flies"
ADJ: "fruit"

""").char_cfg(.99, ignore='[ ]?'), tol=1e-100))

In [ ]:
''.join(fruit.sample())

In [ ]:
#fruit.cfg.language(20).project(lambda x: ''.join(x))

In [ ]:
fruit('fruit flies like a banana ' + EOS)

In [ ]:
prompt = 'The following is a favorite sentence among linguists:'
token_trie_approx = TokenTrieApproximation(llm, fruit)
tracer = TraceSWOR()
for _ in range(1):
    with tracer:
        print('----------------------------------')
        print(tracer.root.mass)
        ys = token_trie_approx.sample(prompt, max_tokens=50, draw=tracer)
        print(ys)